In [1]:
import marimo as mo
import polars as pl
import os
import altair as alt
import statsmodels.formula.api as smf

In [2]:
N_BINS = 5

In [3]:
df_reversal = (
    pl.read_parquet("data/stock_returns.parquet")
    .sort("ticker", "date")
    .with_columns(
        pl.col("return")
        .log1p()
        .rolling_sum(21)
        .shift(1)
        .over("ticker")
        .alias("reversal")
    )
    .drop_nulls()
)

df_reversal

date,ticker,return,reversal
date,str,f64,f64
2020-08-26,"""A""",0.009164,0.011015
2020-08-27,"""A""",-0.002296,0.039601
2020-08-28,"""A""",0.008578,0.019757
2020-08-31,"""A""",0.006327,0.041942
2020-09-01,"""A""",0.005463,0.042533
…,…,…,…
2025-12-22,"""ZTS""",0.012804,0.043132
2025-12-23,"""ZTS""",-0.0021,0.066244
2025-12-24,"""ZTS""",0.016149,0.011889


In [4]:
labels = [str(i) for i in range(N_BINS)]

df_bins = df_reversal.with_columns(
    pl.col("reversal").qcut(N_BINS, labels=labels).over("date").alias("bin")
)

df_bins

date,ticker,return,reversal,bin
date,str,f64,f64,cat
2020-08-26,"""A""",0.009164,0.011015,"""1"""
2020-08-27,"""A""",-0.002296,0.039601,"""2"""
2020-08-28,"""A""",0.008578,0.019757,"""2"""
2020-08-31,"""A""",0.006327,0.041942,"""2"""
2020-09-01,"""A""",0.005463,0.042533,"""2"""
…,…,…,…,…
2025-12-22,"""ZTS""",0.012804,0.043132,"""2"""
2025-12-23,"""ZTS""",-0.0021,0.066244,"""2"""
2025-12-24,"""ZTS""",0.016149,0.011889,"""1"""


In [5]:
df_portfolios = (
    df_bins.group_by("date", "bin")
    .agg(pl.col("return").mean())
    .sort("date", "bin")
    .pivot(index="date", on="bin", values="return")
    .with_columns(pl.col("0").sub(str(N_BINS - 1)).alias("spread (0-4)"))
    .unpivot(index="date", variable_name="portfolio", value_name="return")
    .sort("date", "portfolio")
)

df_portfolios

date,portfolio,return
date,str,f64
2020-08-26,"""0""",-0.004637
2020-08-26,"""1""",-0.00137
2020-08-26,"""2""",-0.002107
2020-08-26,"""3""",-0.002303
2020-08-26,"""4""",0.000579
…,…,…
2025-12-29,"""1""",0.001051
2025-12-29,"""2""",0.00029
2025-12-29,"""3""",-0.002676


In [6]:
df_cumulative_returns = df_portfolios.select(
    "date",
    "portfolio",
    pl.col("return")
    .log1p()
    .cum_sum()
    .mul(100)
    .over("portfolio")
    .alias("cumulative_return"),
)

df_cumulative_returns

date,portfolio,cumulative_return
date,str,f64
2020-08-26,"""0""",-0.464816
2020-08-26,"""1""",-0.137051
2020-08-26,"""2""",-0.210898
2020-08-26,"""3""",-0.230526
2020-08-26,"""4""",0.057891
…,…,…
2025-12-29,"""1""",72.333057
2025-12-29,"""2""",59.056273
2025-12-29,"""3""",65.268696


In [7]:
(
    alt.Chart(df_cumulative_returns)
    .mark_line()
    .encode(
        x=alt.X("date", title=""),
        y=alt.Y("cumulative_return", title="Cumulative Log Returns (%)"),
        color=alt.Color("portfolio", title="Portfolio"),
    )
)

MaxRowsError: The number of rows in your dataset (8052) is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")

Or, see https://altair-viz.github.io/user_guide/large_datasets.html for additional information
on how to plot large datasets.

alt.Chart(...)

In [8]:
df_summary = (
    df_portfolios.group_by("portfolio")
    .agg(
        pl.col("return").mean().mul(252 * 100).alias("mean"),
        pl.col("return").std().mul(pl.lit(252).sqrt() * 100).alias("stdev"),
    )
    .with_columns(pl.col("mean").truediv(pl.col("stdev")).alias("sharpe"))
    .with_columns(pl.exclude("portfolio").round(2))
    .sort("portfolio")
)

df_summary

portfolio,mean,stdev,sharpe
str,f64,f64,f64
"""0""",20.76,22.14,0.94
"""1""",15.16,17.77,0.85
"""2""",12.44,16.41,0.76
"""3""",13.54,15.99,0.85
"""4""",10.42,17.24,0.6
"""spread (0-4)""",10.34,17.21,0.6


In [9]:
df_factors = (
    pl.read_parquet("data/etf_returns.parquet")
    .sort("ticker")
    .pivot(index="date", on="ticker", values="return")
    .sort("date")
)

df_factors

date,MTUM,QUAL,SPY,USMV,VLUE
date,f64,f64,f64,f64,f64
2020-07-28,-0.000458,-0.00547,-0.00625,-0.002256,0.002994
2020-07-29,0.002291,0.008089,0.012209,0.008697,0.003456
2020-07-30,0.000914,-0.005242,-0.003688,-0.004139,-0.009706
2020-07-31,0.002588,0.00613,0.007938,0.000173,-0.009959
2020-08-03,0.015258,0.006948,0.006816,0.003463,0.012294
…,…,…,…,…,…
2025-12-22,0.008998,0.004978,0.006157,0.006909,0.007542
2025-12-23,0.003591,0.001551,0.004674,0.000369,-0.002544
2025-12-24,0.004875,0.004146,0.003431,0.003007,0.007213


In [10]:
df_all_joined = (
    df_portfolios.join(other=df_factors, on="date", how="left")
    .rename({"return": "portfolio_return"})
    .with_columns(pl.exclude("date", "portfolio").mul(100))
    .sort("date", "portfolio")
)

results_list = []
for portfolio_name in df_all_joined["portfolio"].unique().sort():
    df_portfolio = df_all_joined.filter(pl.col("portfolio").eq(portfolio_name))

    formula = "portfolio_return ~ MTUM + QUAL + SPY + USMV + VLUE"
    model = smf.ols(formula=formula, data=df_portfolio)
    result = model.fit()

    portfolio_results = pl.DataFrame(
        {
            "portfolio": portfolio_name,
            "parameter": result.params.index.tolist(),
            "B": result.params.values.tolist(),
            "T": result.tvalues.values.tolist(),
        }
    )

    results_list.append(portfolio_results)

df_regression_results = (
    pl.concat(results_list)
    .pivot(index="portfolio", on="parameter", values=["B", "T"])
    .with_columns(pl.exclude("portfolio").round(2))
)

df_regression_results

portfolio,B_Intercept,B_MTUM,B_QUAL,B_SPY,B_USMV,B_VLUE,T_Intercept,T_MTUM,T_QUAL,T_SPY,T_USMV,T_VLUE
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""0""",0.01,0.06,0.12,0.44,0.19,0.4,0.62,2.05,1.72,5.19,4.03,14.0
"""1""",-0.0,-0.05,0.03,0.37,0.45,0.34,-0.05,-3.05,0.77,7.19,15.39,19.46
"""2""",-0.01,-0.1,0.01,0.36,0.49,0.33,-0.94,-7.6,0.36,9.11,21.89,24.88
"""3""",-0.0,-0.16,-0.01,0.44,0.43,0.34,-0.22,-10.85,-0.31,9.9,17.26,22.65
"""4""",-0.02,-0.22,-0.02,0.71,0.07,0.37,-1.05,-9.62,-0.35,10.24,1.86,15.93
"""spread (0-4)""",0.03,0.28,0.14,-0.26,0.12,0.03,0.99,6.63,1.35,-2.09,1.7,0.74
